In [1]:
# Run startup commands
# Note: if cwd is already the companion repo, this will fail.
# Either restart the kernel first or don't run this cell, in that case.
project_dir=$(pdm info --where)
source $project_dir/.bashnbrc

(ins):
(ins):
(ins):
(ins):


In [2]:
# Make sure we're in the repo directory and git config is correct
GITHUB_URL='https://github.com/edsq/eeskew-pwg-test-000'  # needed for embed-repo-link
tmp_dir="_tmp_pwg_presentation_02-08-2023_part2"
repo_name="eeskew-pwg-test-000"
cd $project_dir/repos &&
rm -rf $tmp_dir &&
mkdir -p $tmp_dir/$repo_name
git clone $repo_name $tmp_dir/$repo_name &&
cd $tmp_dir/$repo_name

Cloning into '_tmp_pwg_presentation_02-08-2023_part2/eeskew-pwg-test-000'...
done.


In [3]:
# Start at the last checkpoint from Part 1
git checkout 461b5ee

Note: checking out '28b2959'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 28b2959 Fix formatting with black


In [4]:
# Create the local venv and install the project (since it was already installed in part 1)
pdm venv create --force python &&
pdm install

⠴ Creating virtualenv using virtualenv.....
Virtualenv 
/Users/Ed/python/eds-notes/repos/_tmp_pwg_presentation_02-08-2023_part2/eeskew-p
wg-test-000/.venv is created successfully
⠋ Fetching hashes for resolved packages...
Synchronizing working set with lock file: 7 to add, 0 to update, 0 to remove

  ⠋ Installing black 23.1.0...
  ⠋ Installing black 23.1.0...
  ⠋ Installing click 8.1.3... 
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing cowsay 5.0...           
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing cowsay 5.0...           
  ⠋ Installing mypy-extensions 1.0.0...
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing cowsay 5.0...           
  ⠋ Installing mypy-extensions 1.0.0...
  ⠋ Installing packaging 23.0...       
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing cowsay 5.0...           
  ⠋ Ins

In [5]:
# For debugging, ensure we're in the right place and using the right venv/interpreter
pdm info

PDM version:
  2.4.8
Python Interpreter:
  /Users/Ed/python/eds-notes/repos/_tmp_pwg_presentation_02-08-2023_part2/eeskew
-pwg-test-000/.venv/bin/python (3.11)
Project Root:
  /Users/Ed/python/eds-notes/repos/_tmp_pwg_presentation_02-08-2023_part2/eeskew
-pwg-test-000
Project Packages:
  None


# Publishing on (Test)PyPI

Part 2 of notes from a talk given to the WSU Python Working Group on February 8, 2023.  See Part 1 [here](part1.ipynb).

In this chapter, we'll cover the basics of publishing a project on PyPI or TestPyPI using PDM,
including managing relevant project metadata.

This tutorial is primarily aimed at macOS and Linux users, although the commands for Windows should mostly translate.

:::{note}
A companion repository with the example project created in these notes is available [here](https://github.com/edsq/eeskew-pwg-test-000).
:::

## Package version

Right now, the package version (`"0.1.0"`) is stored in the `pyproject.toml` file (in the `project.version` keyword).  Python packages typically specify their version in a `__version__.py` file.

In [6]:
# This cell hidden in presentation and docs
echo '__version__ = "0.1.0"' > src/eeskew_pwg_test_000/__version__.py

In [7]:
# This cell hidden in presentation and docs
cat << EOF > pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
EOF

In [8]:
# checkpoint
git-checkpoint

Previous HEAD position was 28b2959 Fix formatting with black
HEAD is now at e3473bd Use dynamic versioning in pyproject.toml


In [9]:
embed-repo-link "Create the file {src/eeskew_pwg_test_000/__version__.py}, and add the \`__version__\` variable to it.  The new file should look like:"

Create the file [`src/eeskew_pwg_test_000/__version__.py`](https://github.com/edsq/eeskew-pwg-test-000/tree/e3473bd9f4fb41856362e1ee36116a4ab38adcd3/src/eeskew_pwg_test_000/__version__.py), and add the `__version__` variable to it. The new file should look like:


In [10]:
simple-diff HEAD~ src/eeskew_pwg_test_000/__version__.py | show-code python

```python
# src/eeskew_pwg_test_000/__version__.py
__version__ = "0.1.0"
```


### Single-sourcing the package version

Now, our package version appears in both `__version__.py` and `pyproject.toml`.  To keep our code [DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself) and avoid these two potentially getting out of sync, we'd like `__version__.py` to be the single source of truth for our package version.

In [11]:
embed-repo-link "We can achieve this by modifying {pyproject.toml} so that the version is [dynamic metadata](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/#dynamic):"

We can achieve this by modifying [`pyproject.toml`](https://github.com/edsq/eeskew-pwg-test-000/tree/e3473bd9f4fb41856362e1ee36116a4ab38adcd3/pyproject.toml) so that the version is [dynamic metadata](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/#dynamic):


In [12]:
simple-diff --context 2 HEAD~ pyproject.toml | show-code toml

```toml
# pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.dev-dependencies]
dev = [

...


[project]
name = "eeskew-pwg-test-000"
description = "A test project for presentation to the WSU Python Working Group."

...

readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[build-system]

...
```


Note the new `tool.pdm.version` table, that the `project.version` key is gone, and the new `project.dynamic` array.

:::{note}
We could have equivalently left `pyproject.toml` unchanged, and used [`importlib.metadata`](https://docs.python.org/3/library/importlib.metadata.html#distribution-versions) to get the version in `__version__.py` from `pyproject.toml` as follows:

```python
# src/eeskew_pwg_test_000/__version__.py
from importlib.metadata import version

__version__ = version("eeskew-pwg-test-000")
```

Whether you set the version in `pyproject.toml` or in `__version__.py` is essentially a matter of taste (although the way I've written the `test-publish` script below will only work with the latter).  The most important thing is that you only set it in one or the other.
:::

We can check the version like so:

In [13]:
pdm show --version eeskew-pwg-test-000

0.1.0


In [14]:
# This cell hidden in presentation and docs
echo 'from eeskew_pwg_test_000.__version__ import __version__' > src/eeskew_pwg_test_000/__init__.py

In [15]:
# checkpoint
git-checkpoint

Previous HEAD position was e3473bd Use dynamic versioning in pyproject.toml
HEAD is now at 609bd67 Add __version__ to __init__


In [16]:
embed-repo-link "We should also add the \`__version__\` variable to {src/eeskew_pwg_test_000/__init__.py}:"

We should also add the `__version__` variable to [`src/eeskew_pwg_test_000/__init__.py`](https://github.com/edsq/eeskew-pwg-test-000/tree/609bd67f8cfe674db8f00181f220027a032648b9/src/eeskew_pwg_test_000/__init__.py):


In [17]:
simple-diff HEAD~ src/eeskew_pwg_test_000/__init__.py | show-code python

```python
# src/eeskew_pwg_test_000/__init__.py
from eeskew_pwg_test_000.__version__ import __version__
```


This allows us to check the version from python in the conventional way:

In [18]:
pdm run python -c "import eeskew_pwg_test_000; print(eeskew_pwg_test_000.__version__)"

0.1.0


:::{seealso}
See the [PDM docs on dynamic versioning](https://pdm.fming.dev/latest/pyproject/build/#dynamic-versioning) for more information.
:::

## Packaging the project

Let's review the project as it exists so far:

In [19]:
# remove pycache to clean up output
rm -r src/eeskew_pwg_test_000/__pycache__

In [20]:
tree

.
├── README.md
├── pdm.lock
├── pyproject.toml
└── src
    └── eeskew_pwg_test_000
        ├── __init__.py
        ├── __version__.py
        └── sarcasm.py

2 directories, 6 files


To create an [sdist](https://packaging.python.org/en/latest/specifications/source-distribution-format/) and [wheel](https://packaging.python.org/en/latest/specifications/binary-distribution-format/):

In [21]:
pdm build

Building sdist...
Built sdist at 
/Users/Ed/python/eds-notes/repos/_tmp_pwg_presentation_02-08-2023_part2/eeskew-p
wg-test-000/dist/eeskew-pwg-test-000-0.1.0.tar.gz
Built wheel at 
/Users/Ed/python/eds-notes/repos/_tmp_pwg_presentation_02-08-2023_part2/eeskew-p
wg-test-000/dist/eeskew_pwg_test_000-0.1.0-py3-none-any.whl


### What did `pdm build` do?

We've created a new directory named `dist`, where these two distribution formats have been placed.

In [22]:
tree

.
├── README.md
├── build
├── dist
│   ├── eeskew-pwg-test-000-0.1.0.tar.gz
│   └── eeskew_pwg_test_000-0.1.0-py3-none-any.whl
├── pdm.lock
├── pyproject.toml
└── src
    └── eeskew_pwg_test_000
        ├── __init__.py
        ├── __version__.py
        └── sarcasm.py

4 directories, 8 files


We could install this project into a different python environment with `python -m pip install dist/eeskew-pwg-test-000-0.1.0.tar.gz` or `python -m pip install dist/eeskew_pwg_test_000-0.1.0-py3-none-any.whl` (the latter is faster).

## Publishing the project

Now we'll publish the project on (Test)PyPI.

:::{note}
To publish on the actual index (PyPI, not TestPyPI), simply replace `testpypi` with `pypi` in the instructions that follow.  Try not to pollute PyPI with throwaway projects!
:::

### Setting up PyPI credentials

0. First, make an account on [TestPyPI](https://test.pypi.org).
1. Navigate to your account settings, scroll down to "API tokens", and click "Add API token"
2. Give the token a descriptive name, set the scope to "Entire account (all projects)", and click "Add token".
3. Copy the token that appears - heeding the warning that it will appear only once!
4. Now we'll configure PDM with these credentials (replacing `<PASTE_YOUR_TOKEN_HERE>` with the token you've just copied):

```bash
pdm config repository.testpypi.username "__token__"
pdm config repository.testpypi.password "<PASTE_YOUR_TOKEN_HERE>"
```

### Setting up a test-publish PDM script

To publish on TestPyPI, we could now simply run:

```bash
pdm publish -r testpypi
```

Note that you do not need to run `pdm build` first - PDM will build the distribution as part of `publish` anyway.

:::{seealso}
See the [PDM docs on publishing to PyPI for more information](https://pdm.fming.dev/latest/usage/project/#publish-the-project-to-pypi).
:::

However, TestPyPI won't let you overwrite an existing version of your package, so we have to bump our version every time we want to do this.  Let's set up a PDM script to automate that.

:::{attention}
If you are publishing on PyPI (*not* TestPyPI), you probably don't want to use this script.  Publishing will be as simple as running `pdm bump {version}` to increment your package version number (for example, `pdm bump patch`), and then publishing with `pdm publish` (equivalent to `pdm publish -r pypyi`).
:::

First, ensure you have the [pdm-bump](https://github.com/carstencodes/pdm-bump) plugin installed.

In [23]:
# This cell hidden in presentation and docs
cat << "EOF" > pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.scripts]
test-publish.shell = '''\
VERSION=$(pdm show --version eeskew-pwg-test-000)
pdm bump patch > /dev/null
BUMPED_VERSION=$(pdm show --version eeskew-pwg-test-000)
DEV_VERSION=$BUMPED_VERSION.dev$(date +%s)
echo "__version__ = \"$DEV_VERSION\"" > src/eeskew_pwg_test_000/__version__.py
pdm publish -r testpypi
echo "__version__ = \"$VERSION\"" > src/eeskew_pwg_test_000/__version__.py
'''

[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
EOF

In [24]:
# checkpoint
git-checkpoint

Previous HEAD position was 609bd67 Add __version__ to __init__
HEAD is now at 21a57db Add test-publish PDM script


In [25]:
embed-repo-link "We add a new PDM script in the \`tool.pdm.scripts\` table of {pyproject.toml}:"

We add a new PDM script in the `tool.pdm.scripts` table of [`pyproject.toml`](https://github.com/edsq/eeskew-pwg-test-000/tree/21a57dbc68ceaf06f98e977c99d05fee503c9edf/pyproject.toml):


In [26]:
simple-diff HEAD~ pyproject.toml | show-code toml

```toml
# pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.scripts]
test-publish.shell = '''\
VERSION=$(pdm show --version eeskew-pwg-test-000)
pdm bump patch > /dev/null
BUMPED_VERSION=$(pdm show --version eeskew-pwg-test-000)
DEV_VERSION=$BUMPED_VERSION.dev$(date +%s)
echo "__version__ = \"$DEV_VERSION\"" > src/eeskew_pwg_test_000/__version__.py
pdm publish -r testpypi
echo "__version__ = \"$VERSION\"" > src/eeskew_pwg_test_000/__version__.py
'''

[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",

...
```


:::{note}
A future release of `pdm-bump` will enable this script to be dramatically simplified, or even entirely unecessary.  See this [GitHub issue](https://github.com/carstencodes/pdm-bump/issues/38#issuecomment-1427082451).
:::

When we run `pdm run test-publish`, this script:
1. Gets the current version with `pdm show --version`
2. Changes the package version to a patch bump of that version with `.dev{date in seconds}` appended.  This is a [developmental release](https://peps.python.org/pep-0440/#developmental-releases) format.
3. Publishes the package on TestPyPI
4. Returns the package version to its original value

Let's run it!

In [27]:
pdm run test-publish

Building sdist...
Built sdist at 
/Users/Ed/python/eds-notes/repos/_tmp_pwg_presentation_02-08-2023_part2/eeskew-p
wg-test-000/dist/eeskew-pwg-test-000-0.1.1.dev1678978289.tar.gz
Built wheel at 
/Users/Ed/python/eds-notes/repos/_tmp_pwg_presentation_02-08-2023_part2/eeskew-p
wg-test-000/dist/eeskew_pwg_test_000-0.1.1.dev1678978289-py3-none-any.whl
Uploading eeskew_pwg_test_000-0.1.1.dev1678978289-py3-none-any.whl
Uploading eeskew-pwg-test-000-0.1.1.dev1678978289.tar.gz━━━━━━━━ 4.8/4.8 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 kB • 00:00 • ?
 100%

:::::{note}
If you want to test installing from TestPyPI, you can do so as follows:

::::{tab-set}

:::{tab-item} PDM

Make a new PDM project, add TestPyPI to the local project indices, then install:
```
pdm config --local pypi.test.url https://test.pypi.org/simple/
pdm add your-package
```
:::

:::{tab-item} venv or conda

Create a new `venv` or `conda` environment, *don't forget to activate it*, then run:

```
python -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ your-package
```
:::

::::

Be sure not to `pip install` into your system or conda base environment!  First make a new virtual environment using PDM, venv, or conda, and test your install there.
:::::

:::{seealso}
See the [PDM docs on PDM scripts](https://pdm.fming.dev/latest/usage/scripts/) for more information on writing PDM scripts.
:::

## Other project metadata

There is quite a bit of project metadata we can specify in `pyproject.toml` - here are some other examples.

:::{seealso}
See the [PyPA specification](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/) for a complete list of possible metadata keys.
:::

### Add an entry point to the package

If we're developing a command-line application, we want our users to be able to run the application with a single command, not something like `python path_to_script/script.py`.  We can enable this by adding an [entry point](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/#entry-points) to `pyproject.toml`.

In [28]:
# This cell hidden in presentation and docs
cat << "EOF" > src/eeskew_pwg_test_000/cli.py
import argparse

from eeskew_pwg_test_000.sarcasm import sarcastic_cowsay


def main():
    """Cowsay something sarcastically from the command line."""
    parser = argparse.ArgumentParser()
    parser.add_argument("speech")
    args = parser.parse_args()

    s = args.speech
    sarcastic_cowsay(s)
EOF

In [29]:
# checkpoint
git-checkpoint

Previous HEAD position was 21a57db Add test-publish PDM script
HEAD is now at d53d416 Add cli.py


In [30]:
embed-repo-link "First, add a new module, {src/eeskew_pwg_test_000/cli.py}":

First, add a new module, [`src/eeskew_pwg_test_000/cli.py`](https://github.com/edsq/eeskew-pwg-test-000/tree/d53d4161677336d995c0a8bf5b046740a8a975b4/src/eeskew_pwg_test_000/cli.py):


In [31]:
simple-diff HEAD~ src/eeskew_pwg_test_000/cli.py | show-code python

```python
# src/eeskew_pwg_test_000/cli.py
import argparse

from eeskew_pwg_test_000.sarcasm import sarcastic_cowsay


def main():
    """Cowsay something sarcastically from the command line."""
    parser = argparse.ArgumentParser()
    parser.add_argument("speech")
    args = parser.parse_args()

    s = args.speech
    sarcastic_cowsay(s)
```


In [32]:
# This cell hidden in presentation and docs
cat << "EOF" > pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.scripts]
test-publish.shell = '''\
VERSION=$(pdm show --version eeskew-pwg-test-000)
pdm bump patch > /dev/null
BUMPED_VERSION=$(pdm show --version eeskew-pwg-test-000)
DEV_VERSION=$BUMPED_VERSION.dev$(date +%s)
echo "__version__ = \"$DEV_VERSION\"" > src/eeskew_pwg_test_000/__version__.py
pdm publish -r testpypi
echo "__version__ = \"$VERSION\"" > src/eeskew_pwg_test_000/__version__.py
'''

[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[project.scripts]
sarcasticow = "eeskew_pwg_test_000.cli:main"

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
EOF

In [33]:
# checkpoint
git-checkpoint

Previous HEAD position was d53d416 Add cli.py
HEAD is now at 0b1a2be Add sarcasticow entry point to pyproject.toml


In [34]:
embed-repo-link "Now let's add the script to {pyproject.toml}, in the \`project.scripts\` table":

Now let's add the script to [`pyproject.toml`](https://github.com/edsq/eeskew-pwg-test-000/tree/0b1a2beda38666ee3bac7069ebe2e36d3eeaf636/pyproject.toml), in the `project.scripts` table:


In [35]:
simple-diff HEAD~ pyproject.toml | show-code toml

```toml
# pyproject.toml
...

license = {text = "MIT"}
dynamic = ["version"]

[project.scripts]
sarcasticow = "eeskew_pwg_test_000.cli:main"

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
```


Note the new entry for `sarcasticow` in the `project.scripts` table.

To use the `sarcasticow` command, we need to re-install the project:

In [36]:
pdm install

⠋ Fetching hashes for resolved packages...
All packages are synced to date, nothing to do.
Installing the project as an editable package...
  ✔ Update eeskew-pwg-test-000 0.1.0 -> 0.1.0 successful
36m⠦ Updating eeskew-pwg-test-000 0.1.0 -> 0.1.0...
🎉 All complete!
36m⠦ Updating eeskew-pwg-test-000 0.1.0 -> 0.1.0...



Now we can run our command from within the environment:

In [38]:
pdm run sarcasticow "I'm a sarcastic cow"

  ___________________
| i'm a sArCaStIc cOw |
                   \
                    \
                      ^__^
                      (oo)\_______
                      (__)\       )\/\
                          ||----w |
                          ||     ||


:::{note}
Because we're using `pdm` for environment management, we still need to use `pdm run` to access the script installed into the virtual environment.  We could also first activate the environment with `eval $(pdm venv activate)`.

Unlike the `test-publish` PDM script we wrote earlier, if we activate the environment, we could simply use the `sarcasticow` command by itself (without `pdm run`), and users who install our package with `pip` into their own virtualenv or conda environment will also have access to `sarcasticow`.  Even better, users who install our package through [`pipx`](https://pypa.github.io/pipx/) will be able to use the `sarcasticow` command without activating a virtual environment.
:::

### Updating README

Thus far, we've left our README as an empty file.  This is bad.

In [39]:
# This cell hidden in presentation and docs
cat << "EOF" > README.md
# eeskew-pwg-test-000

This is a companion repository to the presentation [Project Management and Publishing with PDM](https://edsq.github.io/eds-notes/python_notes/pwg_presentation/intro.html).

Command-line usage:

```
$ sarcasticow "I'm a sarcastic cow"

  ___________________
| i'm a sArCaStIc cOw |
  ===================
                   \
                    \
                      ^__^
                      (oo)\_______
                      (__)\       )\/\
                          ||----w |
                          ||     ||

```
EOF

In [40]:
# checkpoint
git-checkpoint

Previous HEAD position was 0b1a2be Add sarcasticow entry point to pyproject.toml
HEAD is now at 21bf650 Add sarcasticow to README


In [41]:
embed-repo-link "Let's update {README.md} to show our utility's usage:"

Let's update [`README.md`](https://github.com/edsq/eeskew-pwg-test-000/tree/21bf6500a6b2d62525f0c22c1e61d175e2a3a9e8/README.md) to show our utility's usage:


In [49]:
simple-diff --comment-fmt '<!-- {} -->' HEAD~ README.md

<!-- README.md -->
# eeskew-pwg-test-000

This is a companion repository to the presentation [Project Management and Publishing with PDM](https://edsq.github.io/eds-notes/python_notes/pwg_presentation/intro.html).

Command-line usage:

```
$ sarcasticow "I'm a sarcastic cow"

  ___________________
| i'm a sArCaStIc cOw |
                   \
                    \
                      ^__^
                      (oo)\_______
                      (__)\       )\/\
                          ||----w |
                          ||     ||

```


### Project URLs

We can add relevant URLs in the [`urls` table](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/#urls), which will appear in the sidebar on PyPI.

In [ ]:
# This cell hidden in presentation and docs
cat << "EOF" > pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.scripts]
test-publish.shell = '''\
VERSION=$(pdm show --version eeskew-pwg-test-000)
pdm bump patch > /dev/null
BUMPED_VERSION=$(pdm show --version eeskew-pwg-test-000)
DEV_VERSION=$BUMPED_VERSION.dev$(date +%s)
echo "__version__ = \"$DEV_VERSION\"" > src/eeskew_pwg_test_000/__version__.py
pdm publish -r testpypi
echo "__version__ = \"$VERSION\"" > src/eeskew_pwg_test_000/__version__.py
'''

[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[project.urls]
Documentation = "https://edsq.github.io/eds-notes/python_notes/pwg_presentation/intro.html"
Repository = "https://github.com/edsq/eeskew-pwg-test-000"

[project.scripts]
sarcasticow = "eeskew_pwg_test_000.cli:main"

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
EOF

In [ ]:
# checkpoint
git-checkpoint

In [ ]:
embed-repo-link "In the \`project.urls\` table of {pyproject.toml}, we set \`Documentation\` to a link to our docs (this Jupyter Book), and \`Repository\` to the GitHub repo for this code."

In [ ]:
simple-diff HEAD~ pyproject.toml | show-code toml

Finally, we publish again to TestPyPI:

In [ ]:
pdm run test-publish

Our page on TestPyPI now shows the README, and the documentation and repository links are available on the sidebar:

:::{image} images/testpypi_project_screenshot.png
:::

## Conclusion

And that's it!  We've gone through the basics of project management, packaging, and publishing on (Test)PyPI.

### Further reading

Some good articles for further reading:

- The official [PyPA packaging tutorial](https://packaging.python.org/en/latest/tutorials/packaging-projects/) is a good place to start.
- I have taken quite a bit of inspiration in my work from the great series [Hypermodern Python](https://cjolowicz.github.io/posts/hypermodern-python-01-setup/) by Claudio Jolowicz.  It goes into much greater detail on various points of project management than I did here, and is well worth a read.
    - Hypermodern Python uses the project manager [Poetry](https://python-poetry.org), not PDM (see my thoughts on that below), but PDM can do nearly everything Poetry can do, so this series is still useful.
- [How to improve Python packaging, or why fourteen tools are at least twelve too many](https://chriswarrick.com/blog/2023/01/15/how-to-improve-python-packaging/) is a good overview of the mess of modern python project management tools.

Some important accepted Python Enhancement Proposals that shape the modern python packaging ecosystem:

- [PEP 621](https://peps.python.org/pep-0621/) (project metadata in `pyproject.toml`)
- [PEP 508](https://peps.python.org/pep-0508/) (dependency specification syntax)
- [PEP 440](https://peps.python.org/pep-0440/) (version numbers for dependency specification)
- [PEP 517](https://peps.python.org/pep-0517/) (specifying the build backend in `pyproject.toml` independently of the build frontend)


### Other project management tools

You'll note that Hypermodern Python recommends using the project manager [Poetry](https://python-poetry.org).  Poetry was a cutting-edge option at the time that series was written, and it has grown to considerable popularity.  I chose not to recommend it in this tutorial for several reasons:

- Poetry is not fully [PEP 517](https://peps.python.org/pep-0517/) compliant.  This means you and your users will not be able to use any other tool to build and install your project.
- By default, Poetry imposes an upper bound on dependency version numbers.  This goes against [Python standards](https://peps.python.org/pep-0440/) and will likely cause problems down the line: see articles [here](https://iscinumpy.dev/post/bound-version-constraints/) and [here](https://iscinumpy.dev/post/poetry-versions/).
- The Poetry developers chose to "deprecate" their old install script by [*causing it to randomly fail*](https://youtu.be/Gr9o8MW_pb0?t=290) instead of sunsetting with a typical deprecation warning.  This bizarre choice raises serious concerns about the future of the application.
- PDM has considerably fewer open issues on GitHub than Poetry: at the time of this writing, [34](https://github.com/pdm-project/pdm/issues) vs [575](https://github.com/python-poetry/poetry/issues).
    - Some of this is explainable by PDM having fewer users than Poetry.  However, PDM has 4.1k stars on Github vs. Poetry's 24.1k, so using those stars as a proxy for the number of users, we see Poetry has 17 times the number of open issues but only 6 times the number of users as PDM.
- The Poetry developers are famously stubborn and slow to implement new python standards, while the PDM developer is remarkably responsive and active in development.
- Poetry is much more inflexible about resolving dependency conflicts than PDM, which quickly becomes a huge headache for complicated projects.

Another project manager worth looking in to is [Hatch](https://hatch.pypa.io/latest/).  I don't have any experience with this particular tool, but it does support modern Python standards and has the benefit of being an official part of the Python Packaging Authority (PyPA).  However, it does not have lockfile support, as there is not yet an established standard (PEP [665](https://peps.python.org/pep-0665/) was an attempt, but it was rejected).  If you don't need a lockfile in your work, it may be worth considering.

Finally, for those using `conda`, [anaconda-project](https://anaconda-project.readthedocs.io/en/latest/) deserves a mention.  It isn't a full-fledged project manager (for example, you can't easily use it to publish on PyPI), but it does dramatically simplify sharing and running code.